In [1]:
import pandas as pd
from ptrail.core.TrajectoryDF import PTRAILDataFrame
from ptrail.preprocessing.filters import Filters as filters
from ptrail.features.kinematic_features import KinematicFeatures
from ptrail.features.temporal_features import TemporalFeatures
from ptrail.preprocessing.interpolation import Interpolation
from ptrail.utilities.conversions import Conversions as con
import ptrail.utilities.constants as const
import datetime as dt

# Reading the gulls dataset and converting to PTRAILDataFrame.
# Also, lets, print the first 5 points of the dataset to
# see how the dataframe looks.
gulls = pd.read_csv('https://raw.githubusercontent.com/YakshHaranwala/PTRAIL/main/examples/data/gulls.csv')
gulls = PTRAILDataFrame(gulls,
                        latitude='location-lat',
                        longitude='location-long',
                        datetime='timestamp',
                        traj_id='tag-local-identifier',
                        rest_of_columns=[])

In [2]:
pdf = pd.read_csv('https://raw.githubusercontent.com/YakshHaranwala/PTRAIL/main/examples/data/geolife_sample.csv')
geolife = PTRAILDataFrame(data_set=pdf,
                             latitude='lat',
                             longitude='lon',
                             datetime='datetime',
                             traj_id='id')


In [3]:
atlantic = pd.read_csv('https://raw.githubusercontent.com/YakshHaranwala/PTRAIL/main/examples/data/atlantic.csv')
atlantic = con.convert_directions_to_degree_lat_lon(atlantic, 'Latitude',"Longitude")
def convert_to_datetime(row):
        this_date = '{}-{}-{}'.format(str(row['Date'])[0:4], str(row['Date'])[4:6], str(row['Date'])[6:])
        this_time = '{:02d}:{:02d}:00'.format(int(row['Time']/100), int(str(row['Time'])[-2:]))
        return '{} {}'.format(this_date, this_time)
atlantic['DateTime'] = atlantic.apply(convert_to_datetime, axis=1)
atlantic = PTRAILDataFrame(atlantic,
                          latitude='Latitude',
                          longitude='Longitude',
                          datetime='DateTime',
                          traj_id='ID',
                          rest_of_columns=[])
atlantic = TemporalFeatures.create_date_column(atlantic)

In [4]:
pdf = pd.read_csv('https://raw.githubusercontent.com/YakshHaranwala/PTRAIL/main/examples/data/starkey.csv')
starkey = PTRAILDataFrame(data_set=pdf,
                          latitude='lat',
                          longitude='lon',
                          datetime='DateTime',
                          traj_id='Id')

In [5]:
t1 = gulls.reset_index().loc[gulls.reset_index()[const.TRAJECTORY_ID] == '91732'][[const.TRAJECTORY_ID, const.DateTime, const.LAT, const.LONG]]
t1 = PTRAILDataFrame(data_set=t1,
                     datetime='DateTime',
                     traj_id='traj_id',
                     latitude='lat',
                     longitude='lon')

In [6]:
def segment_traj_by_week(df):
    """
        Given a dataframe containing trajectory data, segment all
        the trajectories by each week.

        Parameters
        ----------
            df: PTRAILDataFrame
                The dataframe containing trajectory data.

        Returns
        -------
            pandas.core.dataframe.DataFrame:
                The dataframe containing segmented trajectories
                with a new column added called segment_id
    """
    # First, create the date column and get all the unique traj_ids
    # in the dataframe..
    df = TemporalFeatures.create_date_column(df)
    ids_ = list(df.traj_id.value_counts().keys())

    # Get the ideal number of IDs by which the dataframe is to be split.
    df_chunks = []
    for i in range(len(ids_)):
        small_df = df.reset_index().loc[df.reset_index()[const.TRAJECTORY_ID] == ids_[i]]
        df_chunks.append(small_df)

    # Now, iterate over the entire dataframe and then segment
    # the trajectories by 1 week each.
    results = []
    for i in range(len(ids_)):
        # Take the traj_df of a single Trajectory out from the
        # list of chunks and find their max and min timestamps.
        traj = df_chunks[i]
        t_max = traj.reset_index()[const.DateTime].max()
        t_min = traj.reset_index()[const.DateTime].min()

        # For iteration purposes, set t_1 to min and t_2 to
        # t_1 + 7 days.
        t_1 = t_min
        t_2 = t_1 + dt.timedelta(days=7)
        seg_id = 1

        # Now, segment the trajectories into smaller segments
        # wherein each segment contains the points of a span
        # of 7 days only.
        while t_2 < t_max:
            if t_2 < t_max:
                seg = filters.filter_by_date(traj,
                                             start_date=t_1.strftime('%Y-%m-%d'),
                                             end_date=t_2.strftime('%Y-%m-%d'))
                # Once filtered, assign the segment with a segment ID.
                seg['seg_id'] = seg_id

                # Increment the segment id, t_1 and t_2 values by
                # 1, 7 days and 7 days respectively to continue
                # the iteration.
                t_1 += dt.timedelta(days=7)
                t_2 += dt.timedelta(days=7)
                results.append(seg.drop(columns=['index', 'level_0']))

            # If, t_2 is greater than the max time present in the
            # trajectory, then assign t_2 = max and proceed
            # further with segmentation.
            elif t_2 >= t_max:
                seg = filters.filter_by_date(traj,
                                             start_date=t_1.strftime('%Y-%m-%d'),
                                             end_date=t_max.strftime('%Y-%m-%d'))
                # Once filtered, assign the segment with a segment ID.
                seg['seg_id'] = seg_id

                # Increment the segment id, t_1 and t_2 values by
                # 1, 7 days and 7 days respectively to continue
                # the iteration.
                t_1 += dt.timedelta(days=7)
                t_2 += dt.timedelta(days=7)
                results.append(seg.drop(columns=['index', 'level_0']))
            seg_id += 1

    # Finally, concat the dataframes, set the index as
    # [traj_id, seg_id, DateTime].
    return pd.concat(results).reset_index().set_index(['traj_id', 'seg_id', 'DateTime'])

In [7]:
%%time

seg = segment_traj_by_week(starkey)

CPU times: user 2min 35s, sys: 652 ms, total: 2min 36s
Wall time: 2min 36s


In [8]:
print(len(starkey))

287136


In [9]:
%%time

# Now, generating the stats of kinematic features for
# all the segments of the trajectories.

# First, splitting the dataframe into several parts
# wherein each part has a segment of the trajectory only.
ids_ = list(seg.reset_index()['traj_id'].value_counts().keys())

df_chunks = []
for i in range(len(ids_)):
    small_df = seg.reset_index().loc[seg.reset_index()[const.TRAJECTORY_ID] == ids_[i]]
    df_chunks.append(small_df)

final_chunks = []
for i in range(len(ids_)):
    seg_ids = list(df_chunks[i]['seg_id'].value_counts().keys())
    for j in range(len(seg_ids)):
        mini_df = df_chunks[i].loc[df_chunks[i]['seg_id'] == seg_ids[j]]
        final_chunks.append(mini_df)

print(len(final_chunks))

4976
CPU times: user 41.2 s, sys: 16 ms, total: 41.2 s
Wall time: 41.2 s


In [10]:
def segment_features(df):
    """
        Generate all the kinematic features on the smaller segments.

        Parameters
        ----------
            df: PTRAILDataFrame/pd.DataFrame
                The dataframe containing the trajectory segment.

        Returns
        -------
            pd.core.dataframe.DataFrame:
                The dataframe containing the segment kinematic
                feature stats.
    """
    seg_id = final_chunks[i]['seg_id'].iloc[0]
    stat_df = KinematicFeatures.generate_kinematic_stats(df)
    stat_df['seg_id'] = seg_id
    stat_df = stat_df.reset_index().set_index(['traj_id', 'seg_id', 'Columns'])

    return stat_df

In [15]:
%%time
# Now, generating kinematic features and their stats on each of
# the dataframe here.
import progressbar

final_results = []
bar = progressbar.ProgressBar(max_value=progressbar.UnknownLength)
for i in range(len(final_chunks)):
    final_results.append(segment_features(final_chunks[i]))
    bar.update(i)

final_answer = pd.concat(final_results)

| |                                               #| 4975 Elapsed Time: 0:16:24

CPU times: user 7min 26s, sys: 6min 10s, total: 13min 37s
Wall time: 16min 25s


In [17]:
final_answer.head(40)

mean           std  \
traj_id   seg_id Columns                                            
880109D01 1      Acceleration          1.600362e-04  1.088466e-03   
                 Bearing               1.750986e+02  1.051646e+02   
                 Bearing_Rate          1.171464e-02  1.029251e-01   
                 Distance              2.914998e+02  2.839370e+02   
                 Distance_from_start   1.025254e+03  4.617076e+02   
                 Jerk                  4.409422e-07  3.131269e-06   
                 Rate_of_bearing_rate  1.171464e-02  1.029251e-01   
                 Speed                 1.354668e-01  3.995508e-01   
880119D02 1      Acceleration          2.254526e-05  2.255140e-04   
                 Bearing               1.717234e+02  1.024833e+02   
                 Bearing_Rate         -1.038254e-03  1.049304e-01   
                 Distance              1.181248e+02  1.063050e+02   
                 Distance_from_start   4.346468e+02  2.579344e+02   
                 Jerk                  6.331631e-08  7.623935e-07   
                 Rate_of_bearing_rate -1.038254e-03  1.049304e-01   
                 Speed                 6.061299e-02  9.014658e-02   
          2      Acceleration          8.416691e-05  1.114423e-03   
                 Bearing               1.743838e+02  1.053569e+02   
                 Bearing_Rate          4.741775e-03  1.506678e-01   
                 Distance              1.158744e+02  1.047429e+02   
                 Distance_from_start   4.333041e+02  2.673240e+02   
                 Jerk                  6.566782e-07  1.044940e-05   
                 Rate_of_bearing_rate  4.741775e-03  1.506678e-01   
                 Speed                 6.977280e-02  1.653332e-01   
          3      Acceleration          4.060000e-06  7.022555e-05   
                 Bearing               1.546541e+02  1.095864e+02   
                 Bearing_Rate         -1.487298e-03  7.853078e-02   
                 Distance              1.794725e+02  2.294682e+02   
                 Distance_from_start   4.895102e+02  2.474866e+02   
                 Jerk                  4.171469e-09  7.560761e-08   
                 Rate_of_bearing_rate -1.487298e-03  7.853078e-02   
                 Speed                 7.135458e-02  1.071893e-01   
          4      Acceleration          8.781777e-04  1.401964e-02   
                 Bearing               1.627084e+02  1.104710e+02   
                 Bearing_Rate          1.088292e-02  4.437433e-01   
                 Distance              1.062983e+02  1.244459e+02   
                 Distance_from_start   6.443007e+02  1.121377e+02   
                 Jerk                  3.053720e-05  5.018339e-04   
                 Rate_of_bearing_rate  1.088292e-02  4.437433e-01   
                 Speed                 7.798193e-02  4.027918e-01   

                                                min           10%  \
traj_id   seg_id Columns                                            
880109D01 1      Acceleration         -1.353523e-03 -2.075748e-05   
                 Bearing               0.000000e+00  3.569276e+01   
                 Bearing_Rate         -1.784676e-01 -8.469952e-02   
                 Distance              0.000000e+00  4.238471e+01   
                 Distance_from_start   5.984499e+01  2.543090e+02   
                 Jerk                 -4.075974e-06 -1.168146e-08   
                 Rate_of_bearing_rate -1.784676e-01 -8.469952e-02   
                 Speed                 0.000000e+00  7.619797e-03   
880119D02 1      Acceleration         -4.194241e-04 -2.843878e-05   
                 Bearing               0.000000e+00  4.381483e+01   
                 Bearing_Rate         -6.716448e-01 -9.562361e-02   
                 Distance              0.000000e+00  2.992252e+01   
                 Distance_from_start   2.992252e+01  8.476971e+01   
                 Jerk                 -1.674557e-06 -2.208970e-08   
                 Rate_of_bearing_rate -6.716